<a href="https://colab.research.google.com/github/YimingXu1/DS-notes/blob/main/Projects/U.S.%20Patent%20Phrase%20to%20Phrase%20Matching/PatentSBERTa_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip3 install datasets
!pip3 install transformers
!pip3 install sentencepiece
!pip3 install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel

os.environ["WANDB_DISABLED"] = "true"

In [22]:
class CFG:
    input_path = './drive/MyDrive/Kaggle/USPatent/uspatent/'
    #model_path = './drive/MyDrive/Kaggle/deberta-v3-small/'
    model_path =   '/content/drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned-4/' #'./drive/MyDrive/Kaggle/USPatent/Models/'
    
    learning_rate = 2e-5
    weight_decay = 0.01
    num_fold = 5
    epochs = 5
    batch_size =128
    max_input_length = 130
    batch_size = 64
    num_workers = 2

In [20]:
test_df = pd.read_csv(f"{CFG.input_path}test.csv")
titles = pd.read_csv('./drive/MyDrive/Kaggle/USPatent/uspatent/titles.csv')
test_df = test_df.merge(titles, left_on='context', right_on='code')

# ====================================================
# CPC Data
# ====================================================
cpc_texts = torch.load("./drive/MyDrive/Kaggle/folds-dump-the-two-paths-fix/cpc_texts.pth")
test_df['context_text'] = test_df['context'].map(cpc_texts)
test_df['text'] = test_df['anchor'] + '[SEP]' + test_df['target'] + '[SEP]'  + test_df['context_text']
test_df['text'] = test_df['text'].apply(str.lower)
display(test_df.head())

,id,anchor,target,context,code,title,section,class,subclass,group,main_group,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS. OPTICS,opc drum[sep]inorganic photoconductor drum[sep...
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS. OPTICS,generate in layer[sep]generate by layer[sep]ph...
2,7aa5908a77a7ec24,el display,illumination,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS. OPTICS,el display[sep]illumination[sep]physics. optics
3,09e418c93a776564,adjust gas flow,altering gas flow,F23,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES,F,23.0,NaN,NaN,NaN,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow[sep]altering gas flow[sep]mech...
4,36baf228038e314b,lower trunnion,lower locating,B60,B60,VEHICLES IN GENERAL,B,60.0,NaN,NaN,NaN,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion[sep]lower locating[sep]performi...


In [23]:
tokenizer_deberta_v3 = AutoTokenizer.from_pretrained(CFG.model_path)

In [31]:
class TestDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_length):
        self.text = df['text'].values.astype(str)
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        inputs = self.text[item]
        
        inputs = self.tokenizer(inputs,
                    max_length=self.max_input_length,
                    padding='max_length',
                    truncation=True )
        return torch.as_tensor(inputs['input_ids'], dtype=torch.long),\
               torch.as_tensor(inputs['attention_mask'], dtype=torch.long)

In [41]:
class Custom_Bert_Simple(nn.Module):
    def __init__(self, model_path):
        super().__init__()
        
        config = AutoConfig.from_pretrained(model_path)
        config.num_labels = 1
        self.base = AutoModelForSequenceClassification.from_config(config=config)
        dim = config.hidden_size
        self.dropout = nn.Dropout(p=0)
        self.cls = nn.Linear(dim,1)
        
    def forward(self, input_ids, attention_mask, labels=None):
        base_output = self.base(input_ids=input_ids,
                                attention_mask=attention_mask )

        output = base_output[0]
        if labels is None:
            return output
        
        else:
            return (nn.MSELoss()(torch.squeeze(output,1),labels), output)

In [42]:
def valid_fn(valid_loader, model, device):
    model.eval()
    preds = []
    labels = []
    for step, batch in enumerate(valid_loader):
        input_ids, attention_mask = [i.to(device) for i in batch]
        with torch.no_grad():
            y_preds = model(input_ids, attention_mask)
        preds.append(y_preds.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [43]:
predictions = []
MMscaler = MinMaxScaler()
te_dataset = TestDataset(test_df, tokenizer_deberta_v3, CFG.max_input_length)
te_dataloader = DataLoader(te_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
for fold in tqdm(range(CFG.num_fold)):
    
    model = Custom_Bert_Simple(CFG.model_path)
    #model.load_state_dict(torch.load('/content/drive/MyDrive/Kaggle/USPatent/PatentSBERTa-finetuned-4/pytorch_model.bin')['model'])
    model.to('cuda')
    
    outputs = valid_fn(te_dataloader, model, 'cuda')
    prediction = outputs.reshape(-1)
    predictions.append(MMscaler.fit_transform(prediction.reshape(-1,1)).reshape(-1))
    

100%|██████████| 5/5 [00:13<00:00,  2.66s/it]


In [44]:
predictions = np.mean(predictions, axis=0)
predictions = np.where(predictions<=0, 0, predictions)
predictions = np.where(predictions>=1, 1, predictions)
submission = pd.DataFrame({
    'id': test_df['id'],
    'score': predictions,
})

submission

,id,score
0,4112d61851461f60,0.630501
1,5203a36c501f1b7c,0.538823
2,7aa5908a77a7ec24,0.612674
3,09e418c93a776564,0.666489
4,36baf228038e314b,0.486129
5,b892011ab2e2cabc,0.561536
6,1f37ead645e7f0c8,0.547231
7,71a5b6ad068d531f,0.409371
8,16ae4b99d3601e60,0.526343
9,474c874d0c07bd21,0.631634


In [30]:
submission

,id,anchor,target,context,code,title,section,class,subclass,group,main_group,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS. OPTICS,opc drum[sep]inorganic photoconductor drum[sep...
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS. OPTICS,generate in layer[sep]generate by layer[sep]ph...
2,7aa5908a77a7ec24,el display,illumination,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS. OPTICS,el display[sep]illumination[sep]physics. optics
3,09e418c93a776564,adjust gas flow,altering gas flow,F23,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES,F,23.0,NaN,NaN,NaN,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow[sep]altering gas flow[sep]mech...
4,36baf228038e314b,lower trunnion,lower locating,B60,B60,VEHICLES IN GENERAL,B,60.0,NaN,NaN,NaN,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion[sep]lower locating[sep]performi...
5,b892011ab2e2cabc,carry by platform,carry on platform,B60,B60,VEHICLES IN GENERAL,B,60.0,NaN,NaN,NaN,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,carry by platform[sep]carry on platform[sep]pe...
6,1f37ead645e7f0c8,cap component,upper portion,D06,D06,TREATMENT OF TEXTILES OR THE LIKE; LAUNDERING;...,D,6.0,NaN,NaN,NaN,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component[sep]upper portion[sep]textiles; ...
7,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,H04,ELECTRIC COMMUNICATION TECHNIQUE,H,4.0,NaN,NaN,NaN,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation[sep]artificial neural netwo...
8,16ae4b99d3601e60,transmit to platform,direct receiving,H04,H04,ELECTRIC COMMUNICATION TECHNIQUE,H,4.0,NaN,NaN,NaN,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,transmit to platform[sep]direct receiving[sep]...
9,474c874d0c07bd21,dry corn,dry corn starch,C12,C12,BIOCHEMISTRY; BEER; SPIRITS; WINE; VINEGAR; MI...,C,12.0,NaN,NaN,NaN,HEMISTRY; METALLURGY. BIOCHEMISTRY; BEER; SPIR...,dry corn[sep]dry corn starch[sep]hemistry; met...
